## Stream attributes from Web events using Signals

This notebook creates a new attribute group using the SDK, tests it on the atomic events table and applies.

### Flow of data

```mermaid
flowchart LR
    sp(Snowplow Pipeline)
    stream[/Stream processing/]
    signals(Signals)

    sp --> stream
    stream --> signals
```

---

# Installation and setup

In [ ]:
%pip install snowplow-signals

In [ ]:
from snowplow_signals import Signals
import os

try:
    from google.colab import userdata
    sp_signals = Signals(
            api_url=userdata.get('SP_API_URL'),
            api_key=userdata.get('SP_API_KEY'),
            api_key_id=userdata.get('SP_API_KEY_ID'),
            org_id=userdata.get('SP_ORG_ID'),
        )
except ImportError:
    sp_signals = Signals(
        api_url=os.environ['SP_API_URL'],
        api_key=os.environ['SP_API_KEY'],
        api_key_id=os.environ['SP_API_KEY_ID'],
        org_id=os.environ['SP_ORG_ID'],
)

### Define a new attribute

This block creates a single attribute definition including the logic how it should be calculated (it's filters and aggregation).

The attribute calculates the number of add to cart ecommerce events.

In [ ]:
from snowplow_signals import (
    Attribute,
    Event,
    AtomicProperty,
)

count_page_pings = Attribute(
    name="count_page_pings",
    type="int32",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_ping",
            version="1-0-0",
        )
    ],
    aggregation="counter",
)

visited_pages = Attribute(
    name="visited_pages",
    type="string_list",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0",
        )
    ],
    aggregation="unique_list",
    property=AtomicProperty(
        name="page_title",
    ),
)

### Wrapping the attribute in an attribute group

All attributes need to be included in attribute groups that can be considered as "tables" of attributes.

Attribute groups are immutable and versioned.

In [ ]:
from snowplow_signals import StreamAttributeGroup, session_attribute_key

attribute_group = StreamAttributeGroup(
    name="my_web_attributes",
    version=1,
    attribute_key=session_attribute_key,
    attributes=[
        count_page_pings,
        visited_pages,
    ],
)

### Testing the attribute group

Execute the attribute group on the last one hour of data from the atomic events table to verify that it works correctly.

In [ ]:
data = sp_signals.test(
    attribute_group=attribute_group,
    app_ids=["website"],
)
data

### Applying the attribute group to Signals

The following block pushes the attribute group definition to the Signals API and makes it available for processing.

In [ ]:
published = sp_signals.publish([attribute_group])
print(f"{len(published)} objects published")